# Calculating simple monthly means of some weather data
Original Data: Daily mean ensemble 1980-2022 in 0.25deg resolution from Copernicus.eu, E-OBS dataset

## Presettings

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import os
import re

import xarray as xr

In [ ]:
# All directories
root       = "C:/Users/freiste/OneDrive - Ilmatieteen laitos/Documents/IIASA YSSP 2023"
this_dir   =  os.getcwd()

input_dir  = f"{root}/02 - Data/EU/Copernicus_E-OBS_Weather_Original" # to save space, not all data might be there anymore
output_dir = f"{root}/02 - Data/EU/Copernicus_E-OBS_Weather_Postprocessed"

## Methods

In [ ]:
def import_aggr_and_concat_years(path, files, aggregate='mean'):
    
    for f, file in enumerate(files): 

        # Import file
        data_temp = xr.open_dataset(f"{path}/{file}")
        
        
        # Calc monthly mean to save memory
        var = list(data_temp.data_vars)[0]
        if aggregate=='mean':
            data_temp = data_temp[var].resample(time='1M').mean()
            
        elif aggregate=='sum':
            data_temp = data_temp[var].resample(time='1M').sum(dim='time')
        

        # Concatenate all the years into one variable
        if f==0:
            data = data_temp
        else:
            data = xr.concat((data, data_temp), dim='time')

        data_temp.close()

    return data


In [ ]:
def workflow(input_file_identifier, input_dir, output_filename, aggregate='mean'):
    
    next_regex = re.compile(fr'{input_file_identifier}')
    next_files = [path.name for path in os.scandir(input_dir) if path.is_file() if next_regex.search(path.name)]

    next_data  = import_aggr_and_concat_years(input_dir, next_files, aggregate=aggregate)

    next_data.to_netcdf(f"{output_dir}/{output_filename}")
    
    
    return 'Export successful.'
    

# Execution

In [ ]:
workflow('windspeed_dailymean'    , input_dir, 'windspeed_monthlymean_europe_1980-2022_0.25deg.nc')
workflow('humidity_dailymean'     , input_dir, 'humidity_monthlymean_europe_1980-2022_0.25deg.nc')
workflow('radiation_dailymean'    , input_dir, 'radiation_monthlymean_europe_1980-2022_0.25deg.nc')
workflow('precipitation_dailymean', input_dir, 'precipitation_monthlymean_europe_1980-2022_0.25deg.nc')


In [ ]:
workflow('tempavg_dailymean', input_dir, 'tempavg_monthlymean_europe_1980-2022_0.25deg.nc', aggregate='mean')
workflow('tempavg_dailymean', input_dir, 'tempavg_monthlydegreedays_europe_1980-2022_0.25deg.nc', aggregate='sum')

workflow('tempmin_daily', input_dir, 'tempmin_monthlymean_europe_1980-2022_0.25deg.nc', aggregate='mean')
workflow('tempmin_daily', input_dir, 'tempmin_monthlydegreedays_europe_1980-2022_0.25deg.nc', aggregate='sum')

workflow('tempmax_daily', input_dir, 'tempmax_monthlymean_europe_1980-2022_0.25deg.nc', aggregate='mean')
workflow('tempmax_daily', input_dir, 'tempmax_monthlydegreedays_europe_1980-2022_0.25deg.nc', aggregate='sum')